In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ds-w7/sample_submit.csv
/kaggle/input/2022-ds-w7/valid/dog/dog.12347.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12386.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12383.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12387.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12390.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12380.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12393.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12351.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12348.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12358.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12370.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12377.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12397.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12353.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12388.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12372.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12356.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12382.jpg
/kaggle/input/2022-ds-w7/valid/dog/dog.12369.jpg
/kaggle/input/2022-ds-w7/v

In [2]:
# import keras
# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from keras.applications import mobilenet
# from keras.optimizer_v1 import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
# from keras.applications.vgg16 import preprocess_input
# from keras.applications.vgg16 import decode_prediction
# from keras.applications.vgg16 import VGG16
# from keras.applications import 

In [3]:
# dataroot="/kaggle/input/2022-ds-w7/"
# csvpath=os.path.join(dataroot,"Label2Names.csv")
# df_data=pd.read_csv(csvpath,header=None)
# dataroottraindog=os.path.join(dataroot,"train/dog")
# dataroottestdog=os.path.join(dataroot,"train/dog")
# dataroottraincat=os.path.join(dataroot,"train/cat")
# dataroottestcat=os.path.join(dataroot,"train/cat")

submit=pd.read_csv('/kaggle/input/2022-ds-w7/sample_submit.csv')

In [4]:
# train_ds = keras.utils.image_dataset_from_directory(
#     directory='training_data/',
#     labels='inferred',
#     label_mode='categorical',
#     batch_size=32,
#     image_size=(256, 256))
# validation_ds = keras.utils.image_dataset_from_directory(
#     directory='validation_data/',
#     labels='inferred',
#     label_mode='categorical',
#     batch_size=32,
#     image_size=(256, 256))

In [5]:
train_path = '/kaggle/input/2022-ds-w7/train'
valid_path = '/kaggle/input/2022-ds-w7/valid'
test_path = '/kaggle/input/2022-ds-w7/test'

train_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(train_path,
                                                        target_size=(224,224),
                                                        batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(valid_path,
                                                        target_size=(224,224),
                                                        batch_size=30)
test_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(test_path,
                                                        target_size=(224,224),
                                                        batch_size=50,
                                                        shuffle=False,
                                                        class_mode=None)

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 451 images belonging to 1 classes.


In [6]:
#텐서에 탑제
import tensorflow as tf
tf.device("/device:GPU:0")

2022-11-07 04:19:04.590400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 04:19:04.686785: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 04:19:04.687756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 04:19:04.690687: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
base_model = vgg16.VGG16(weights = 'imagenet', include_top=False,
                        input_shape=(224,224,3))

58900480/58889256 [==============================] - 0s 0us/step


In [8]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
last_layer = base_model.get_layer('block5_pool')
last_output = last_layer.output

x = Flatten()(last_output)

x = Dense(64, activation='relu', name='FC_2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax', name='softmax')(x)

new_model=Model(inputs=base_model.input, outputs=x)
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
from tensorflow.keras import optimizers
optimizer=optimizers.Adam(learning_rate=0.0001)


new_model.compile(optimizer, loss='categorical_crossentropy',
                 metrics=['accuracy'])

new_model.fit_generator(train_batches, steps_per_epoch=4,
                       validation_data=valid_batches, validation_steps=2,
                       epochs=25, verbose=2)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-11-07 04:19:08.179724: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-11-07 04:19:09.836446: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


4/4 - 12s - loss: 0.9096 - accuracy: 0.5750 - val_loss: 4.2986 - val_accuracy: 0.5833
Epoch 2/25
4/4 - 1s - loss: 0.6004 - accuracy: 0.7750 - val_loss: 1.4997 - val_accuracy: 0.7333
Epoch 3/25
4/4 - 1s - loss: 0.5600 - accuracy: 0.6875 - val_loss: 0.9201 - val_accuracy: 0.8167
Epoch 4/25
4/4 - 1s - loss: 0.8092 - accuracy: 0.6500 - val_loss: 0.5539 - val_accuracy: 0.8833
Epoch 5/25
4/4 - 1s - loss: 0.3759 - accuracy: 0.9250 - val_loss: 0.2640 - val_accuracy: 0.9167
Epoch 6/25
4/4 - 1s - loss: 0.3408 - accuracy: 0.8500 - val_loss: 0.1577 - val_accuracy: 0.9333
Epoch 7/25
4/4 - 1s - loss: 0.1648 - accuracy: 0.9062 - val_loss: 0.1147 - val_accuracy: 0.9333
Epoch 8/25
4/4 - 1s - loss: 0.2683 - accuracy: 0.9500 - val_loss: 0.1424 - val_accuracy: 0.9333
Epoch 9/25
4/4 - 1s - loss: 0.2311 - accuracy: 0.9000 - val_loss: 0.0782 - val_accuracy: 0.9667
Epoch 10/25
4/4 - 1s - loss: 0.1288 - accuracy: 0.9250 - val_loss: 0.0846 - val_accuracy: 0.9667
Epoch 11/25
4/4 - 1s - loss: 0.1895 - accuracy: 0

In [13]:
expect=new_model.predict(test_batches)

In [14]:
np.argmax(expect,axis=1)

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,

In [15]:
submit.label=np.argmax(expect,axis=1)

In [16]:
submit

,id,label
0,0,0
1,1,1
2,2,1
3,3,1
4,4,0
...,...,...
446,446,0
447,447,0
448,448,0
449,449,0


In [17]:
submit.to_csv('submission.csv',index=False)